Universidad del valle de Guatemala
Andres Quinto
Andree Toledo
Laboratorio 5 DS / Analisis de sentimientos

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import io 
import wordcloud as wc
import random
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.util import ngrams
from nltk import word_tokenize
from collections import Counter, defaultdict

In [ ]:
#Colocamos los valores en lowercase para que no haya problemas con las mayusculas
df = pd.read_csv('train.csv')

df['text'].str.lower()
df['location'].str.lower()
df['keyword'].str.lower()
#print(df.head())

print(df)

In [ ]:
#Eliminacion de caracteres especiales y signos de puntuacion
df.text = df.text.str.replace('[#,@,&,(,),!,?,/,{,},%,!]', '')
df.location = df.location.str.replace('[#,@,&,(,),!,?,/,{,},%,!]', '')
df.keyword = df.keyword.str.replace('[#,@,&,(,),!,?,/,{,},%,!]', '')
#Eliminacion de url
df.text = [re.sub('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*','',i) for i in df.text]
df.text = [re.sub(r'^https?:\/\/.*[\r\n]*', '', i) for i in df.text]
#Eliminacion de emoticonos
df.text = [re.sub('[^a-zA-Z0-9 ]+','', i) for i in df.text]


In [ ]:
print(df)

In [ ]:
#Eliminacion de conjunciones (stopwords)
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from cgitb import text

#https://towardsdatascience.com/how-to-clean-text-data-639375414a2f 

stopwords = set(stopwords.words('english') + ['the', 'i', 'a', 'deeds', 'im','rt','jk','btw','lol','yolo','lmao','lmfao','fb','like','get','em', 'I', 'The', 'A', 'Amp', 'amp'])
expresiones = ['im','rt','jk','btw','lol','yolo','lmao','lmfao','fb','like','get','em', 'Im', 'in', 'In', '2']
for i in expresiones:
    stopwords.add(i)

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
# Creation of the colum named 'text without stopwords'
df['text_without_stopwords'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
# print(df.text)
clean_tweets = df['text_without_stopwords']
print(df['text_without_stopwords'])


#Obtencion de la frecuencia de las palabras tanto de los tweets de desastres como de los que no.

In [ ]:
frec_dic_tweets = {}
for word in clean_tweets:
    if word in frec_dic_tweets:
        frec_dic_tweets[word] += 1
    else:
        frec_dic_tweets[word] = 1

tweets_frecuency = pd.DataFrame.from_dict(frec_dic_tweets, orient='index')
tweets_frecuency


Que palabra se repite mas?

In [ ]:
#Creacion de una lista de palabras para cada tweet
word_count = Counter()

for title in clean_tweets:
    word_count.update(word.strip('.,?!"\'').lower() for word in title.split())
#A continuacion se crea una lista con los 10 tweets mas frecuentes
word_count.most_common(10)

In [ ]:
cloud_tweet = WordCloud(background_color="white",width=1000,height=1000, max_words=10,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(word_count)
plt.imshow(cloud_tweet)

#Grafico de barras para las palabras mas repetidas

In [ ]:
datos_tweet = word_count.most_common(10)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x_axis = [i[0] for i in datos_tweet]
y_axis = [i[1] for i in datos_tweet]
ax.bar(x_axis,y_axis)
plt.show()

N-Grama

In [ ]:
global_data = df['text_without_stopwords']
x = len(global_data)


In [ ]:
val=round(x*0.1,0)
random_sample=random.sample(list(global_data),int(val))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
digrama=[]
print(digrama)
size=2
def ngram(ngrama, size):
  for word in range(len(random_sample)):
    try:
      for item in ngrams(random_sample[word].split(),size):
          ngrama.append(item)
    except:
        return

ngram(digrama, size)
print(digrama)

In [ ]:
trigrama=[]
print(trigrama)
size=3
ngram(trigrama, size)
print(trigrama)

In [ ]:
tetragrama=[]
print(tetragrama)
size=4
ngram(tetragrama, size)
print(tetragrama)

In [ ]:
pentgrama=[]
print(pentgrama)
size=5
ngram(pentgrama, size)
print(pentgrama)

In [ ]:
#Referencia de como realizar un analisis de sentimientos
#https://www.kaggle.com/rohithreddy/analyzing-sentiments-of-tweets-using-nltk
#https://www.arsys.es/blog/analisis-sentimientos-python-jupyter-notebooks

#Realizamos una funcion para tokenizar los tweets
def get_tweets(texts):
    corpus = []
    for text in texts:
        corpus.extend(word_tokenize(text))
    return corpus

# clasificamos los twees como desastres o no desastres
disaster_tweet = get_tweets(df[df["target"] == 1]["text"])
non_disaster_tweet = get_tweets(df[df["target"] == 0]["text"])


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

SX = SentimentIntensityAnalyzer()

sentimentValue = np.zeros(len(df['text']))
for k in range(len(df['text'])):
    sentimentValue[k] = SX.polarity_scores(df['text'][k]).get('compound')
df.insert(loc=len(df.columns), column = 'sentimentValue', value = sentimentValue)

In [ ]:
# media del dataset
# sentimentValue es -0.15. 
print("Media de sentimiento: ", df['sentimentValue'].mean())

# Los 10 tweets con mayor sentimiento positivo
print(df.sort_values(by='sentimentValue').head(10)['text'])

# Los 10 tweets con menor sentimiento negativo
print(df.sort_values(by='sentimentValue', ascending=False).head(10)['text'])

# Promedio de sentimentValue para tweets relacionados a desastre y no desastre.
# La media de los tweets sobre desastre es -0.27.
print("Media sentimentValue desastre: ", df[df["target"] == 1]['sentimentValue'].mean())
# La media de los tweets sobre no desastre es -0.06. 
print("Media sentimentValue no desastre: ", df[df["target"] == 0]['sentimentValue'].mean())